In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR1 = '/content/drive/My Drive/Mali'
BASE_DIR = '/home/sandesh/Documents/Minor Project/predicting-poverty-replication'

RANDOM_SEED = 7 # for reproducibility
PROCESSED_DIR = os.path.join(BASE_DIR1, 'processed')
# these relate to training the CNN to predict nightlights
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR1, 'cnn_images')
CNN_SAVE_DIR = '/content/drive/My Drive/Mali/models/trained_model.pt'

In [ ]:
df_download = pd.read_csv(os.path.join(PROCESSED_DIR, 'image_download_locs.csv'))
downloaded = os.listdir(os.path.join(BASE_DIR1, 'mali_2015', 'images'))

In [ ]:
df_download['row'] = np.arange(len(df_download))

In [ ]:
downloaded =  list(set(downloaded).intersection(set(df_download['image_name'])))
idx_not_download = df_download.set_index('image_name').drop(downloaded).reset_index()
idx_not_download = list(idx_not_download['image_name'])
idx_not_download = list(set(idx_not_download).intersection(set(df_download['image_name'])))
df_download.set_index('image_name').drop(idx_not_download).reset_index()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin,row
0,10.270931292294023_-6.135291430593985_10.31584...,10.270931,-6.135291,10.315847,-6.105348,2.159553,0.0,mli,0,0
1,10.285903213696015_-6.135291430593985_10.31584...,10.285903,-6.135291,10.315847,-6.105348,2.159553,0.0,mli,0,1
2,10.345790899303983_-6.120319509191993_10.31584...,10.345791,-6.120320,10.315847,-6.105348,2.159553,0.0,mli,0,2
3,10.300875135098007_-6.105347587790001_10.31584...,10.300875,-6.105348,10.315847,-6.105348,2.159553,0.0,mli,0,3
4,10.330818977901991_-6.105347587790001_10.31584...,10.330819,-6.105348,10.315847,-6.105348,2.159553,0.0,mli,0,4
...,...,...,...,...,...,...,...,...,...,...
14746,18.249884438603985_-2.7668763507580074_18.2199...,18.249884,-2.766876,18.219941,-2.781848,6.460836,0.0,mli,0,14755
14747,18.264856360005975_-2.7668763507580074_18.2199...,18.264856,-2.766876,18.219941,-2.781848,6.460836,0.0,mli,0,14756
14748,18.23491251720199_-2.7519044293560158_18.21994...,18.234913,-2.751904,18.219941,-2.781848,6.460836,0.0,mli,0,14757
14749,18.189996752996016_-2.7369325079540237_18.2199...,18.189997,-2.736933,18.219941,-2.781848,6.460836,0.0,mli,0,14758


In [ ]:
df_download.drop('row', axis=1, inplace=True)

In [ ]:
# model = Net()
# model.load_state_dict(torch.load(CNN_SAVE_DIR))
# model.eval()

In [ ]:
# the distribution
(df_download['nightlights_bin']==0).mean(), (df_download['nightlights_bin']==1).mean(), (df_download['nightlights_bin']==2).mean()


(0.752710027100271, 0.22018970189701897, 0.02710027100271003)

In [ ]:
df_download.reset_index(drop=True, inplace=True)

In [ ]:
df_download.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin
0,10.270931292294023_-6.135291430593985_10.31584...,10.270931,-6.135291,10.315847,-6.105348,2.159553,0.0,mli,0
1,10.285903213696015_-6.135291430593985_10.31584...,10.285903,-6.135291,10.315847,-6.105348,2.159553,0.0,mli,0
2,10.345790899303983_-6.120319509191993_10.31584...,10.345791,-6.120320,10.315847,-6.105348,2.159553,0.0,mli,0
3,10.300875135098007_-6.105347587790001_10.31584...,10.300875,-6.105348,10.315847,-6.105348,2.159553,0.0,mli,0
4,10.330818977901991_-6.105347587790001_10.31584...,10.330819,-6.105348,10.315847,-6.105348,2.159553,0.0,mli,0


In [ ]:
df_download['is_train'] = True

In [ ]:
np.random.seed(RANDOM_SEED)
groups = df_download.groupby(['cluster_lat', 'cluster_lon'])
for _, g in groups:
    n_ims = len(g)
    n_train = int(0.8 * n_ims)
    n_valid = n_ims - n_train
    valid_choices = np.random.choice(np.arange(n_ims), replace=False, size=n_valid).tolist()
    current_index = g.index
    idx_valid = current_index[valid_choices]
    df_download['is_train'].loc[idx_valid] = False

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
df_download['is_train'].mean()

0.8

In [ ]:
# save this new dataframe
df_download.to_csv(os.path.join(PROCESSED_DIR, 'image_download_actual.csv'), index=False)

In [ ]:
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid'), exist_ok=True)

labels = ['0', '1', '2']
for l in labels:
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l), exist_ok=True)
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', l), exist_ok=True)

In [ ]:
t = df_download[df_download['is_train']]
v = df_download[~df_download['is_train']]

In [ ]:
len(t), len(v)

(11808, 2952)

In [ ]:
# # uses symlinking to save disk space
# print('copying train images')
# for im_name, nl, country in tqdm(zip(t['image_name'], t['nightlights_bin'], t['country']), total=len(t)):
#     country_dir = None
#     if country == 'mli':
#         country_dir = 'mali_2015'
#     else:
#         print(f"no match for country {country}")
#         raise ValueError()
#     src = os.path.abspath(os.path.join(BASE_DIR1, country_dir, 'images', im_name))
#     dest = os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', str(nl), im_name)
#     if os.symlink(src, dest, target_is_directory = False) != None:
#         print("error creating symlink")
#         raise ValueError()

# print('copying valid images')
# for im_name, nl, country in tqdm(zip(v['image_name'], v['nightlights_bin'], v['country']), total=len(v)):
#     country_dir = None
#     if country == 'mli':
#         country_dir = 'mali_2015'
#     else:
#         print(f"no match for country {country}")
#         raise ValueError()
#     src = os.path.abspath(os.path.join(BASE_DIR1, country_dir, 'images', im_name))
#     dest = os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(nl), im_name)
#     if os.symlink(src, dest,target_is_directory = False) != None:
#         print("error creating symlink")
#         raise ValueError()

In [ ]:
# shows count distribution in train folder, make sure this matches above
counts = []
for l in ['0', '1', '2']:
    counts.append(len(os.listdir(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[8892, 2610, 320]
[0.752156995432245, 0.22077482659448486, 0.027068177973270175]
11822


# **Train Model**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optimTrain Model 
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
model_ft = torch.load(CNN_SAVE_DIR)
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [ ]:
data_dir = CNN_TRAIN_IMAGE_DIR
input_size = 224
num_classes = 3
batch_size = 8
num_epochs = 25
feature_extract = True

In [ ]:
data_dir = CNN_TRAIN_IMAGE_DIR
input_size = 224
num_classes = 3
batch_size = 8
num_epochs = 25
feature_extract = True

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")
# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
print(image_datasets)
print(os.path.join(data_dir, 'train'))
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}
print(dataloaders_dict['train'])
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Initializing Datasets and Dataloaders...
{'train': Dataset ImageFolder
    Number of datapoints: 11822
    Root location: /content/drive/My Drive/Mali/cnn_images/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ), 'valid': Dataset ImageFolder
    Number of datapoints: 2968
    Root location: /content/drive/My Drive/Mali/cnn_images/valid
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )}
/content/drive/My Drive/Mali/cnn_images/train
device: cuda:0


In [ ]:
model_ft = model_ft.to(device)

In [ ]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        if epoch > 10:
            # fine tune whole model
            for param in model_ft.parameters():
                param.requires_grad = True
            optimizer = optim.SGD(model_ft.parameters(), lr=1e-4, momentum=0.9)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            print(dataloaders[phase])
            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
criterion = nn.CrossEntropyLoss()
model_ft, hist  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/24
----------



train Loss: 0.5953 Acc: 0.7637



valid Loss: 0.5313 Acc: 0.7800

Epoch 1/24
----------



train Loss: 0.5419 Acc: 0.7806



valid Loss: 0.5150 Acc: 0.7884

Epoch 2/24
----------


...


valid Loss: 0.3745 Acc: 0.8282

Epoch 23/24
----------



train Loss: 0.4035 Acc: 0.8274



valid Loss: 0.3853 Acc: 0.8332

Epoch 24/24
----------



train Loss: 0.3893 Acc: 0.8371



valid Loss: 0.3699 Acc: 0.8325

Training complete in 76m 6s
Best val Acc: 0.833221


In [ ]:
path = '/content/drive/My Drive/Mali/models/mali_trained_model'
path = os.path.join(path, 'mali_trained_model1.pt')
assert not os.path.isfile(path), print('A model is already saved at this location')
print(f'Saving model to {path}')
torch.save(model_ft, path)

Saving model to /content/drive/My Drive/Mali/models/mali_trained_model/mali_trained_model1.pt


In [ ]:
# you can run below if you want to see the final accuracy on nightlights over the train set
model_ft.eval()   # Set model to evaluate mode

criterion = nn.CrossEntropyLoss()
running_loss = 0.0
running_corrects = 0
total = 0

# Iterate over data.
for inputs, labels in tqdm(dataloaders_dict['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    total += len(preds)
        
print(running_corrects.double()/total)


tensor(0.8520, device='cuda:0', dtype=torch.float64)
